# DAMR
Hi, here is the code of DAMR (Dynamic Ajacency Matrix Representation) 
for data imputation in COVID-19 dataset.

# 1. Import library

In [2]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional

# 2. Data Pre-processing of COVID dataset

In [3]:
#Define hyperparameter
Miss_perc=0.7
Split_perc1=0.8
Split_perc2=0.1


In [4]:
#Generate Missing 3D.
path='Data/COVID/raw'
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)


()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


# Run Baseline

In [5]:
##Lags for variable 1 ,modify input variable here!
COVID_X1_all=COVID_features[:,0,:]
#Transform the shape into Bidirectional problems
COVID_X1_all=series_to_supervised(COVID_X1_all,15,16)
COVID_Y1_all=COVID_X1_all.iloc[:,15*7:15*7+7]
COVID_X1_all_1=COVID_X1_all.iloc[:,:15*7]
COVID_X1_all_2=COVID_X1_all.iloc[:,15*7+7:]
COVID_X1_all=np.hstack((COVID_X1_all_1,COVID_X1_all_2))
COVID_X1_all=np.array(COVID_X1_all)
COVID_X1_all=np.reshape(COVID_X1_all,(len(COVID_X1_all),COVID_features.shape[2],30))
#Input 1->Groundtruth
COVID_features=COVID_features[15:-15,:,:]#(473, 3, 7)
print(COVID_features.shape)
#Input2->Groundtruth
print(COVID_X1_all.shape)#(473,7,30)
COVID_Y1_all=np.array(COVID_Y1_all)
COVID_Y1_all=np.reshape(COVID_Y1_all,(len(COVID_Y1_all),7,1))
#Input3->Groundtruth
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
#Output1->Groundtruth
print(COVID_Y1_all.shape)#(473, 7, 1)

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)


In [6]:
##Input and output in DAMR!
# print(COVID_features_aftslid.shape)#473*3*7
# print(COVID_X1_all_aftslid.shape)#473*7*30
# print(COVID_Y1_all.shape)#(473, 7, 1)
# print(Adj_dist.shape)#7*7

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
#Generate Missing 3D.
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)->Groundtruth
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)#(402, 3, 7)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)#(50, 3, 7)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)#(51, 3, 7)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)#(503,3,7)




()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [7]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

     var1(t-30)  var2(t-30)  var3(t-30)  var4(t-30)  var5(t-30)  var6(t-30)  \
40    -0.999750   -0.999640   -0.999998   -0.998295   -0.999455   -1.000000   
41    -0.999732   -0.999602   -0.999997   -0.998460   -0.999610   -0.999831   
42    -0.999414   -0.999559   -0.999995   -0.998428   -0.999580   -0.999831   
43    -0.999572   -0.998144   -0.999996   -0.998391   -0.999163   -0.999831   
44    -0.999545   -0.998938   -0.999993   -0.998385   -0.999299   -0.999505   
..          ...         ...         ...         ...         ...         ...   
498    0.877529    0.912044    0.897712    0.669947    0.753512    0.930475   
499    0.883936    0.912813    0.845386    0.651245    0.773144    0.932053   
500    0.877445    0.941876    0.849615    0.654818    0.756063    0.936982   
501    0.891917    0.946000    0.863293    0.658358    0.757690    0.951261   
502    0.894732    0.933138    0.868913    0.697081    0.759260    0.939355   

     var7(t-30)  var1(t-29)  var2(t-29)  var3(t-29)

In [8]:
type(Sliding_out_var1_sup)

pandas.core.frame.DataFrame

# 3.Dynamic Adjacency matrix layer

In [9]:
#3.1 Distance-based adjacency matrix 
#Country: BZ CL ID IR MX PR SA.
COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7

In [10]:
data1=pd.DataFrame([[-51.9253,-71.543,78.9629],[53.688,-102.5528,-75.0152]])
data2=pd.DataFrame([[-31.9253,-41.543,38.9629],[43.688,62.5528,65.0152]])
Global_MeanMI(data1,data2,2)
data1=pd.DataFrame(data1)
data2=pd.DataFrame(data2)
data1=np.array(data1).reshape(-1)
data2=np.array(data2).reshape(-1)

In [11]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7
print(MeanMI)

(473, 3, 7)
(49,)
(7, 7)
[[1.         0.62222485 0.22861592 0.14298509 0.4207043  0.57563845
  0.49251353]
 [0.63107997 1.         0.19598218 0.35797114 0.48721059 0.6004145
  0.59416752]
 [0.         0.         1.         0.         0.         0.
  0.        ]
 [0.19971669 0.38248321 0.33925041 1.         0.34084987 0.25441679
  0.29492926]
 [0.38018332 0.46596633 0.         0.23505277 1.         0.65964495
  0.78745477]
 [0.58567339 0.60704203 0.09411415 0.14830608 0.65557119 1.
  0.79269602]
 [0.48214303 0.60106569 0.04810739 0.18647903 0.78191082 0.80339824
  1.        ]]


In [12]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7
print(ConcatMI)

(473, 3, 7)
[[1.         0.62222485 0.22861592 0.14298509 0.4207043  0.57563845
  0.49251353]
 [0.63107997 1.         0.19598218 0.35797114 0.48721059 0.6004145
  0.59416752]
 [0.         0.         1.         0.         0.         0.
  0.        ]
 [0.19971669 0.38248321 0.33925041 1.         0.34084987 0.25441679
  0.29492926]
 [0.38018332 0.46596633 0.         0.23505277 1.         0.65964495
  0.78745477]
 [0.58567339 0.60704203 0.09411415 0.14830608 0.65557119 1.
  0.79269602]
 [0.48214303 0.60106569 0.04810739 0.18647903 0.78191082 0.80339824
  1.        ]]


In [13]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [14]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(473, 3, 7)


C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))


[[[0.59257427 0.77995    0.44905877 ... 0.39913382 0.37992352 0.27213659]
  [0.64839184 0.56524566 0.34485495 ... 0.27951428 0.27961504 0.93946545]
  [0.45711272 0.70808083 0.65290474 ... 0.34611487 0.29100813 0.69925615]
  ...
  [0.5209933  0.58587361 0.32593881 ... 0.43896657 0.65002032 0.37337298]
  [0.63416912 0.74292381 0.98380393 ... 0.48848416 0.66661665 0.58162541]
  [0.54226172 0.60220297 0.59936802 ... 0.23597635 0.8087062  0.31313313]]

 [[0.5720576  0.43753843 0.84113799 ... 0.42594857 0.48326062 0.43492169]
  [0.50060792 0.60548095 0.50838057 ... 0.60339276 0.44620901 0.58129156]
  [0.99657866 0.92175912 0.25860627 ... 0.62568863 0.73693375 0.81572985]
  ...
  [0.3169035  0.48679054 0.73912487 ... 0.62370448 0.53590375 0.35683646]
  [0.3817399  0.42671935 0.74666854 ... 0.91201159 0.651379   0.8215176 ]
  [0.74124526 0.6835567  0.199903   ... 0.48681986 0.68218346 0.2021132 ]]

 [[0.50496839 0.3183058  0.40207439 ... 0.89950303 0.39329078 0.37865952]
  [0.48772736 0.522565

D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not mis

In [15]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done

C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
C:\Users\77154\Desktop\put into git\model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.

[[[0.59257427 0.77995    0.44905877 ... 0.39913382 0.37992352 0.27213659]
  [0.64839184 0.56524566 0.34485495 ... 0.27951428 0.27961504 0.93946545]
  [0.45711272 0.70808083 0.65290474 ... 0.34611487 0.29100813 0.69925615]
  ...
  [0.5209933  0.58587361 0.32593881 ... 0.43896657 0.65002032 0.37337298]
  [0.63416912 0.74292381 0.98380393 ... 0.48848416 0.66661665 0.58162541]
  [0.54226172 0.60220297 0.59936802 ... 0.23597635 0.8087062  0.31313313]]

 [[0.5720576  0.43753843 0.84113799 ... 0.42594857 0.48326062 0.43492169]
  [0.50060792 0.60548095 0.50838057 ... 0.60339276 0.44620901 0.58129156]
  [0.99657866 0.92175912 0.25860627 ... 0.62568863 0.73693375 0.81572985]
  ...
  [0.3169035  0.48679054 0.73912487 ... 0.62370448 0.53590375 0.35683646]
  [0.3817399  0.42671935 0.74666854 ... 0.91201159 0.651379   0.8215176 ]
  [0.74124526 0.6835567  0.199903   ... 0.48681986 0.68218346 0.2021132 ]]

 [[0.50496839 0.3183058  0.40207439 ... 0.89950303 0.39329078 0.37865952]
  [0.48772736 0.522565

D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Software\Installation\lib\site-packages\fancyimpute\solver.py:55: UserWarning: Input matrix is not mis

In [16]:
print(DyConcatMI.shape)

(473, 7, 7)


In [17]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=[]
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.         0.29576857 ... 0.14038921 0.12160314 0.10489198]
  [0.09207001 1.         0.26335366 ... 0.35937549 0.3264367  0.3042932 ]
  [0.26503304 0.12969448 1.         ... 0.09442775 0.05460493 0.37277273]
  ...
  [0.25638568 0.34015967 0.2367404  ... 1.         0.15614608 0.12570822]
  [0.26141968 0.35216896 0.2117083  ... 0.183941   1.         0.13697255]
  [0.10123083 0.17128123 0.38315561 ... 0.         0.         1.        ]]

 [[1.         0.         0.29576857 ... 0.14038921 0.12160314 0.10489198]
  [0.09207001 1.         0.26335366 ... 0.35937549 0.3264367  0.3042932 ]
  [0.26503304 0.12969448 1.         ... 0.09442775 0.05460493 0.37277273]
  ...
  [0.25638568 0.34015967 0.2367404  ... 1.         0.15614608 0.12570822]
  [0.26141968 0.35216896 0.2117083  ... 0.183941   1.         0.13697255]
  [0.10123083 0.17128123 0.38315561 ... 0.         0.      

In [18]:
#3.8 FFT-based MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each batch

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(24, 24)
(24, 24)
(24, 7, 7)
(12, 12)
(12, 12)
(12, 7, 7)
(5, 5)
(5, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.         0.29576857 ... 0.14038921 0.12160314 0.10489198]
  [0.09207001 1.         0.26335366 ... 0.35937549 0.3264367  0.3042932 ]
  [0.26503304 0.12969448 1.         ... 0.09442775 0.05460493 0.37277273]
  ...
  [0.25638568 0.34015967 0.2367404  ... 1.         0.15614608 0.12570822]
  [0.26141968 0.35216896 0.2117083  ... 0.183941   1.         0.13697255]
  [0.10123083 0.17128123 0.38315561 ... 0.         0.         1.        ]]

 [[1.         0.         0.29576857 ... 0.14038921 0.12160314 0.10489198]
  [0.09207001 1.         0.26335366 ... 0.35937549 0.3264367  0.3042932 ]
  [0.26503304 0.12969448 1.         ... 0.09442775 0.05460493 0.37277273]
  ...
  [0.25638568 0.34015967 0.2367404  ... 1.         0.15614608 0.12570822]
  [0.26141968 0.35216896 0.2117083  ... 0.183941   

# Begin with DAMR

In [19]:
##Begin with DAMR
print(COVID_features_aftslid.shape)#473*3*7
print(COVID_X1_all_aftslid.shape)#473*7*30
print(COVID_Y1_all.shape)#(473, 7, 1)
print(Adj_dist.shape)#7*7
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

(473, 3, 7)
(463, 7, 30)
(473, 7, 1)
(7, 7)


In [20]:
## Define Model---GAT+DIFFCONV
inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

(None, 7, 30)
(None, 7, 3)
(None, 7, 7)


In [21]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj])

In [22]:
Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
print(Att_weights_P)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7), dtype=tf.float32, name=None), name='tf.math.reduce_mean/Mean:0', description="created by layer 'tf.math.reduce_mean'")


In [23]:
# GCN_output_P =  GCNConv(16, dropout_rate=0.3, 
#                         activation='relu', use_bias=True)([inputs_feat, inputs_adj])

In [24]:
diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, Att_weights_P])#diff_out=nodes*channels

In [25]:
con_out_P_2 = Concatenate(axis=-1)([GAT_output_P,diff_out])
dense_P = Dense(8, activation='relu')(con_out_P_2)

In [26]:
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)



In [27]:
model = Model(inputs=[inputs_ret, inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 30)]      0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 7, 7)]       0           []                               
                                                                                                  
 gat_conv (GATConv)             ((None, 7, 64),      2048        ['input_1[0][0]',                
                                 (None, 7, 4, 7))                 'input_3[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 7, 3)]       0           []                           

In [28]:
#Get 473*7*7 for static adjacency matrix
#3.1 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#3.2 MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#3.3 ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#3.4 FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data
# #3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
# coe1=0.1
# coe2=0.8
# coe3=0.4
# coe4=0.1
# Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
# #3.10 ->()ConcatMI approach
# Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

In [29]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [30]:
# #DyMeanMI
# #3.2 Mean MI-based dynamic adjacency matrix
# print(COVID_features_aftslid.shape)#(473, 3, 7)
# #Index all of the rows and columns
# MeanMI=[]
# for k in range(COVID_features_aftslid.shape[0]):
#     for i in range(COVID_features_aftslid.shape[2]):
#         for j in range(COVID_features_aftslid.shape[2]):
#             MeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
#                                     COVID_features_aftslid[:k,:,j],2))
# DyMeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
# DyMeanMI=DyMeanMI.transpose(2, 0, 1)
# print(DyMeanMI.shape)#(473,7,7)

In [31]:

print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1


(473, 3, 7)
(463, 7, 30)
(473, 7, 1)
(473, 7, 7)


In [32]:
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =72
epochs = 100
seed = 42
verbose = 1
ChangeAdj=Adj_FeatureMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=ChangeAdj[:int(len(ChangeAdj)*0.8),:,:]
adj_val=ChangeAdj[int(len(ChangeAdj)*0.8):int(len(ChangeAdj)*0.9),:,:]
adj_test=ChangeAdj[int(len(ChangeAdj)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

In [33]:
print(x_train.shape)
print(features_train.shape)
print(adj_train.shape)
print(y_train.shape)

(370, 7, 30)
(378, 7, 3)
(378, 7, 7)
(378, 7, 1)


In [34]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1, 
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [x_train, features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=500,
                  batch_size=batch_size, #validation_split=0.1, 
                  validation_data=([x_val, features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error", 
                      monitor="val_loss", patience=20, restore_best_weights=True)],
  )


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


ValueError: Data cardinality is ambiguous:
  x sizes: 370, 378, 378
  y sizes: 378
Make sure all arrays contain the same number of samples.

In [ ]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)   
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([x_test, features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())


In [ ]:
#Get the mask of y_test.
Mask_BZ=pd.read_csv("Data/COVID/10%/BZ_10.csv")
Mask_CL=pd.read_csv("Data/COVID/10%/CL_10.csv")
Mask_ID=pd.read_csv("Data/COVID/10%/ID_10.csv")
Mask_IR=pd.read_csv("Data/COVID/10%/IR_10.csv")
Mask_MX=pd.read_csv("Data/COVID/10%/MX_10.csv")
Mask_PR=pd.read_csv("Data/COVID/10%/PR_10.csv")
Mask_SA=pd.read_csv("Data/COVID/10%/SA_10.csv")

In [ ]:
Mask_BZ=np.array(Mask_BZ)
missing_mask=[]
for i in range(Mask_BZ.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_BZ[i])))
missing_mask=np.array(missing_mask).reshape(Mask_BZ.shape[0],Mask_BZ.shape[1])
missing_mask_BZ_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of BZ
missing_mask_BZ_test1=missing_mask_BZ_test[:,0]

In [ ]:
Mask_CL=np.array(Mask_CL)
missing_mask=[]
for i in range(Mask_CL.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_CL[i])))
missing_mask=np.array(missing_mask).reshape(Mask_CL.shape[0],Mask_CL.shape[1])
missing_mask_CL_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of CL
missing_mask_CL_test1=missing_mask_CL_test[:,0]
Mask_ID=np.array(Mask_ID)
missing_mask=[]
for i in range(Mask_ID.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_ID[i])))
missing_mask=np.array(missing_mask).reshape(Mask_ID.shape[0],Mask_ID.shape[1])
missing_mask_ID_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of ID
missing_mask_ID_test1=missing_mask_ID_test[:,0]
Mask_IR=np.array(Mask_IR)
missing_mask=[]
for i in range(Mask_IR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_IR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_IR.shape[0],Mask_IR.shape[1])
missing_mask_IR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of IR
missing_mask_IR_test1=missing_mask_IR_test[:,0]
Mask_MX=np.array(Mask_MX)
missing_mask=[]
for i in range(Mask_MX.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_MX[i])))
missing_mask=np.array(missing_mask).reshape(Mask_MX.shape[0],Mask_MX.shape[1])
missing_mask_MX_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of MX
missing_mask_MX_test1=missing_mask_MX_test[:,0]
Mask_PR=np.array(Mask_PR)
missing_mask=[]
for i in range(Mask_PR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_PR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_PR.shape[0],Mask_PR.shape[1])
missing_mask_PR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of PR
missing_mask_PR_test1=missing_mask_PR_test[:,0]
Mask_SA=np.array(Mask_SA)
missing_mask=[]
for i in range(Mask_SA.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_SA[i])))
missing_mask=np.array(missing_mask).reshape(Mask_SA.shape[0],Mask_SA.shape[1])
missing_mask_SA_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of SA
missing_mask_SA_test1=missing_mask_SA_test[:,0]

In [ ]:
missing_mask_ALL_test1=np.stack((missing_mask_BZ_test1,missing_mask_CL_test1
            ,missing_mask_ID_test1,missing_mask_IR_test1,
            missing_mask_MX_test1,missing_mask_PR_test1,missing_mask_SA_test1
           ),axis=1)

In [ ]:
#Get test set result.
get_metrics(y_test[missing_mask_ALL_test1].flatten(), test_predicted[missing_mask_ALL_test1].flatten())